## **Part 4: [Exercise]** Rhyme Re-themer Chatbot

Below is a poetry generation example that showcases how you might organize two different tasks under the guise of a single agent. The system calls back to the simple Gradio example, but extends it with some boiler-plate responses and logic behind the scenes.

It's primary feature is as follows:
- On the first response, it will generate a poem based on your response.
- On subsequent responses, it will keep the format and structure of your original rhyme while modifying the topic of the poem.

**Problem:** At present, the system should function just fine for the first part, but the second part is not yet implemented.

**Objective:** Implement the rest of the `rhyme_chat2_stream` method such that the agent is able to function normally.

To make the gradio component easier to reason with, a simplified `queue_fake_streaming_gradio` method is provided that will simulate the gradio chat event loop with the standard Python `input` method

In [ ]:
def rhyme_chat2_stream(message, history, return_buffer=True):
    '''This is a generator function, where each call will yield the next entry'''

    first_poem = None
    for entry in history:
        if entry[0] and entry[1]:
            ## If a generation occurred as a direct result of a user input,
            ##  keep that response (the first poem generated) and break out
            first_poem = entry[1]
            break

    if first_poem is None:
        ## First Case: There is no initial poem generated. Better make one up!

        buffer = "Oh! I can make a wonderful poem about that! Let me think!\n\n"
        yield buffer

        ## iterate over stream generator for first generation
        inst_out = ""
        chat_gen = chain1.stream({"input" : message})
        for token in chat_gen:
            inst_out += token
            buffer += token
            yield buffer if return_buffer else token

        passage = "\n\nNow let me rewrite it with a different focus! What should the new focus be?"
        buffer += passage
        yield buffer if return_buffer else passage

    else:
        ## Subsequent Cases: There is a poem to start with. Generate a similar one with a new topic!

        buffer = f"Sure! Here you go!\n\n"
        yield buffer

        # return  ## <- TODO: Early termination for generators. Comment this out

        ########################################################################
        ## TODO: Invoke the second chain to generate the new rhymes.

        ## iterate over stream generator for second generation
        chat_gen = chain2.stream({"input" : first_poem, "topic" : message})
        for token in chat_gen:
            buffer += token
            yield buffer if return_buffer else token

        ## END TODO
        ########################################################################

        passage = "\n\nThis is fun! Give me another topic!"
        buffer += passage
        yield buffer if return_buffer else passage